# 层和块
## 首先回顾一下多层感知机

In [2]:
import torch
from torch import nn
from torch.nn import functional as F
# nn.Sequential定义了一个特殊的Module
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[-0.2991, -0.1046,  0.0874, -0.2300,  0.0343, -0.1330,  0.0512, -0.0452,
          0.1212, -0.1037],
        [-0.3057,  0.0446,  0.0827, -0.1326,  0.0322, -0.1837,  0.0484, -0.0229,
          0.0047, -0.1052]], grad_fn=<AddmmBackward0>)

![](images/image18.jpg)

In [5]:
# 自定义块
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))
net2 = MLP()
# 值会改变是因为权重发生了变化
net2(X)

tensor([[-0.0760,  0.2792,  0.1522, -0.1889,  0.0976, -0.0613,  0.1386, -0.1603,
          0.1145,  0.1179],
        [-0.1652,  0.3796,  0.1279, -0.1429,  0.0474, -0.2383,  0.0950, -0.1387,
          0.1502,  0.2669]], grad_fn=<AddmmBackward0>)

In [14]:
# 也可以自己定义一个Sequential顺序块
# 在 PyTorch 中，nn.Module 的 _modules 属性默认是一个有序字典，即 collections.OrderedDict。这是因为 PyTorch 0.4.0 版本开始引入的改变，旨在确保模块的顺序与它们添加到父模块中的顺序一致。
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X
net3 = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net3(X)

tensor([[-0.0666, -0.1382, -0.0026,  0.0443, -0.0134, -0.2327, -0.0582, -0.0396,
         -0.0173, -0.0502],
        [ 0.0204, -0.1036, -0.0093, -0.0425, -0.0469, -0.2892, -0.0452,  0.1041,
          0.0687, -0.1320]], grad_fn=<AddmmBackward0>)

In [15]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    def forward(self, X):
        X = self.linear(X)
        x = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
net4 = FixedHiddenMLP()
net4(X)

tensor(0.5593, grad_fn=<SumBackward0>)

In [18]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    def forward(self, X):
        return self.linear(self.net(X))
# 因为NestMLP和FixedHiddenMLP都是继承NestMLP，所以可以放入nn.Sequential
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1293, grad_fn=<SumBackward0>)

In [20]:
import torch
from torch import nn
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0114,  0.2241,  0.3173, -0.0275, -0.0289,  0.2842,  0.3515, -0.3032]])), ('bias', tensor([-0.2076]))])


In [21]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2076], requires_grad=True)
tensor([-0.2076])


In [23]:
# 因为我们还没有做梯度的计算，所以grad为None
net[2].weight.grad == None

True

In [26]:
# 这段代码中的 *[(name, param.shape) for name, param in net[0].named_parameters()] 会生成一个列表，列表中的每个元素都是一个包含两个元素的元组 (name, param.shape)，其中 name 是参数的名称，param.shape 是参数的形状。然后，* 解包运算符将这个列表解包为独立的元组，然后传递给 print 函数，这样 print 函数会分别打印每个元组，显示参数名称和形状。如果不加*，那就生成一个列表，列表中每个元素都是一个元组对
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [27]:
net.state_dict()['2.bias'].data

tensor([-0.2076])

In [28]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}', block1())
    return net
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.4500],
        [0.4500]], grad_fn=<AddmmBackward0>)

In [29]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


## 内置初始化

In [36]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))

In [39]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
# apply函数将对net的每个模块嵌套递归遍历
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 9.9937e-03,  1.4355e-02,  9.5805e-03, -9.2660e-05]), tensor(0.))

In [40]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
# apply函数将对net的每个模块嵌套递归遍历
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [42]:
# 对某些模块使用不同的初始化方法
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.2810, -0.5836, -0.5611,  0.2000])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [51]:
# 自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", [(name, param.shape) for name, param in m.named_parameters()])
        nn.init.uniform_(m.weight, -10, 10)
        # 将小于5的值设为0
        m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init)
net[0].weight[:2]

Init [('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]
Init [('weight', torch.Size([1, 8])), ('bias', torch.Size([1]))]


tensor([[-0.0000, -7.6421,  0.0000, -5.5643],
        [-8.7256,  9.7185, -6.7123, -9.1945]], grad_fn=<SliceBackward0>)

In [52]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -6.6421,  1.0000, -4.5643])

In [55]:
# 参数绑定
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
# 第二层和第四层都是共享参数的（因为都是来源于shared，因此我更新第二层，第四层也会同步被更新，权重始终相同，这就叫做参数绑定
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# 自定义层，自定义层和自定义神经网络本质上没有区别，因为层也是nn.Module的子类

In [56]:
import torch
import torch.nn.functional as F
from torch import nn
# 构造一个没有任何参数的自定义层
class CenterdLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, X):
        return X - X.mean()
layer = CenterdLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [57]:
# 将层作为组件合并到构建更复杂的模型中
net = nn.Sequential(nn.Linear(8, 128), CenterdLayer())
Y = net(torch.rand(4, 8))
# 按道理应该是0，但是浮点数计算会有误差
Y.mean()

tensor(-1.8626e-09, grad_fn=<MeanBackward0>)

In [58]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[ 0.2096, -0.2714, -1.0573],
        [ 0.5972, -1.7719, -0.5476],
        [-0.5743, -0.3484,  0.1147],
        [ 0.5745,  0.3608, -0.0843],
        [ 1.4646,  0.1176,  0.0508]], requires_grad=True)

In [59]:
dense(torch.randn(2, 5))

tensor([[1.5703, 0.0000, 0.0000],
        [0.3100, 1.4364, 0.0000]])

In [60]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.0000],
        [0.9737]])

# 读写文件

In [61]:
# 加载和保存张量
import torch
from torch import nn
from torch.nn import functional as F
x = torch.arange(4)
torch.save(x, 'x-file')
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [64]:
# 存储一个张量列表，再把他们读回内存
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
x2, y2

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [65]:
# 写入或读取从字符串映射到张量的字典
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [67]:
# 加载保存模型参数
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [70]:
# 将模型的参数保存为一个叫做"mlp.params"的文件
torch.save(net.state_dict(), 'mlp.params')

In [71]:
# 把模型参数加载回来，必须先得声明MLP()，才能使用load函数
clone = MLP()
clone.load_state_dict(torch.load("mlp.params"))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [72]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [1]:
!nvidia-smi

Thu Sep 07 08:55:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 442.50       Driver Version: 442.50       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce MX350      WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   47C    P8    N/A /  N/A |     68MiB /  2048MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [1]:
import torch
from torch import nn
torch.device('cpu')

device(type='cpu')

In [2]:
# 第0个GPU
torch.cuda.device('cuda')

In [3]:
# 第1个GPU
torch.cuda.device('cuda:1')

In [4]:
# 查询可用GPU的数量
torch.cuda.device_count()

0

In [5]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu]"""
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cpu'), device(type='cpu'), [device(type='cpu')])

In [6]:
x = torch.tensor([1, 2, 3])
# 默认在cpu上
x.device

device(type='cpu')

In [7]:
# 存储在GPU上
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [8]:
# 在第二张GPU上创建一个随机张量
Y = torch.ones(2, 3, device=try_gpu(1))
Y

tensor([[1., 1., 1.],
        [1., 1., 1.]])

# 我之前安装的pytorch是不支持显卡的！

In [14]:
# 神经网络与GPU
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())
net(X)

tensor([[0.4041],
        [0.4041]], grad_fn=<AddmmBackward0>)

In [15]:
# 确认模型参数存储在同一个GPU上
net[0].weight.data.device

device(type='cpu')

![](images/image19.jpg)

## 推理(inference)是跑个forward，训练是算梯度